# 2. Baseline model

In [2]:
# moving avergae forecast

In [3]:
import sys
print(sys.executable)

/opt/conda/envs/jupyterlab/bin/python3


In [2]:
!python -m venv my-env

In [2]:
import sys
print(sys.executable)

/opt/conda/envs/jupyterlab/bin/python3


In [7]:
import sys
sys.path.append('/opt/conda/lib/python3.7/site-packages/scikit-learn')

In [5]:
import pandas as pd
import numpy as np


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [6]:
# Load the data
train = pd.read_parquet('../data/train.parquet')
test = pd.read_parquet('../data/test.parquet')

In [7]:
train.head()

,day_id,but_num_business_unit,dpt_num_department,turnover
0,2017-09-30,64,127,580.308443
1,2017-09-30,119,127,1512.995918
2,2017-09-30,4,88,668.593556
3,2017-09-30,425,127,0.000000
4,2017-09-30,513,73,0.000000


In [8]:
train['day'] = pd.to_datetime(train['day_id'])
train['week'] = train['day'].dt.isocalendar().week
train['month'] = train['day'].dt.month
train['year'] = train['day'].dt.year

In [9]:
test['day'] = pd.to_datetime(test['day_id'])
test['week'] = test['day'].dt.isocalendar().week
test['month'] = test['day'].dt.month
test['year'] = test['day'].dt.year

In [10]:
train

,day_id,but_num_business_unit,dpt_num_department,turnover,day,week,month,year
0,2017-09-30,64,127,580.308443,2017-09-30,39,9,2017
1,2017-09-30,119,127,1512.995918,2017-09-30,39,9,2017
2,2017-09-30,4,88,668.593556,2017-09-30,39,9,2017
3,2017-09-30,425,127,0.000000,2017-09-30,39,9,2017
4,2017-09-30,513,73,0.000000,2017-09-30,39,9,2017
...,...,...,...,...,...,...,...,...
277714,2012-12-29,131,73,1.461821,2012-12-29,52,12,2012
277715,2012-12-29,237,127,759.283046,2012-12-29,52,12,2012
277716,2012-12-29,129,117,1716.399152,2012-12-29,52,12,2012
277717,2012-12-29,468,127,1307.357057,2012-12-29,52,12,2012


In [11]:
last_week_data = train[train['day_id'] == train['day_id'].max()]

In [12]:
# Define the naive forecast function
def naive_forecast(train, test):
    last_week_data = train[train['day_id'] == train['day_id'].max()]
    predictions = []
    for _, row in test.iterrows():
        but_num_business_unit = row['but_num_business_unit']
        dpt_num_department = row['dpt_num_department']
        last_turnover = last_week_data[(last_week_data['but_num_business_unit'] == but_num_business_unit) & 
                                       (last_week_data['dpt_num_department'] == dpt_num_department)]['turnover'].values[0]
        predictions.append(last_turnover)
    return np.array(predictions)

In [13]:
# Define the moving average forecast function
def moving_average_forecast(train, test, window=4):
    predictions = []
    for _, row in test.iterrows():
        but_num_business_unit = row['but_num_business_unit']
        dpt_num_department = row['dpt_num_department']
        relevant_data = train[(train['but_num_business_unit'] == but_num_business_unit) & 
                              (train['dpt_num_department'] == dpt_num_department)].sort_values(by='day')
        last_n_weeks_data = relevant_data['turnover'].tail(window)
        avg_turnover = last_n_weeks_data.mean()
        predictions.append(avg_turnover)
    return np.array(predictions)

In [14]:
train['day'].max()

Timestamp('2017-09-30 00:00:00')

# split train dataset to evaluate the model

In [15]:
weeks = train[train['day_id'] =='2017-09-30']['week'].max() -7
year = train[train['day_id'] =='2017-09-30']['year'].max() -7

In [16]:
from datetime import datetime, timedelta
validation_time = train['day'].max() - timedelta(weeks=8)

In [17]:
# Walk-Forward Validation to evaluate baseline model

# Split data into training and validation sets
train_data = train[train['day']<=validation_time]  # Use all but the last 8 weeks for training
validation_data = train[train['day']>validation_time] # Last 8 weeks for validation

# baseline model

In [24]:
train_data= train_data.reset_index(drop=True)
validation_data= validation_data.reset_index(drop=True)

In [ ]:
# naive forecast
naive_predictions = naive_forecast(train_data, validation_data)

# moving average forecast
moving_avg_predictions = moving_average_forecast(train_data, validation_data, window=4)


# Add predictions to the test dataset
validation_data['naive_forecast'] = naive_predictions
validation_data['moving_avg_forecast'] = moving_avg_predictions

In [ ]:
predictions_naive = validation_data['naive_forecast'].values
predictions_avg= validation_data['moving_avg_forecast'].values
actuals = validation_data['turnover'].values

In [ ]:
# Evaluate the baseline model
from sklearn.metrics import mean_absolute_error, mean_squared_error


mae = mean_absolute_error(actuals, predictions_naive)
mse = mean_squared_error(actuals, predictions_naive)
rmse = mean_squared_error(actuals, predictions_naive, squared=False)

print(f"MAE: {mae}")
print(f"MSE: {mse}")
print(f"RMSE: {rmse}")

# 3. MVP 

In [27]:
train

,day_id,but_num_business_unit,dpt_num_department,turnover,day,week,month,year
0,2017-09-30,64,127,580.308443,2017-09-30,39,9,2017
1,2017-09-30,119,127,1512.995918,2017-09-30,39,9,2017
2,2017-09-30,4,88,668.593556,2017-09-30,39,9,2017
3,2017-09-30,425,127,0.000000,2017-09-30,39,9,2017
4,2017-09-30,513,73,0.000000,2017-09-30,39,9,2017
...,...,...,...,...,...,...,...,...
277714,2012-12-29,131,73,1.461821,2012-12-29,52,12,2012
277715,2012-12-29,237,127,759.283046,2012-12-29,52,12,2012
277716,2012-12-29,129,117,1716.399152,2012-12-29,52,12,2012
277717,2012-12-29,468,127,1307.357057,2012-12-29,52,12,2012


In [ ]:
import pandas as pd
from prophet import Prophet

# Load the data

# Split data into training and validation sets
train_data = train[train['day']<=validation_time]  # Use all but the last 8 weeks for training
validation_data = train[train['day']>validation_time] # Last 8 weeks for validation

# Prepare the data for Prophet
train['ds'] = train['day']
train = train.rename(columns={'turnover': 'y'})

# Separate forecasts for each store-department combination
forecasts = []

for but_num_business_unit in train['but_num_business_unit'].unique():
    for dpt_num_department in train['dpt_num_department'].unique():
        # Filter data for the current store and department
        store_dept_data = train[(train['but_num_business_unit'] == but_num_business_unit) & (train['dpt_num_department'] == dpt_num_department)]
        
        # Check if there's enough data to train the model
        if len(store_dept_data) > 8:  # Ensure at least some data points
            # Initialize and fit the model
            model = Prophet()
            model.fit(store_dept_data[['ds', 'y']])
            
            # Create a dataframe for future dates
            future = model.make_future_dataframe(periods=8, freq='W')
            
            # Predict future turnover
            forecast = model.predict(future)
            
            # Extract the relevant part of the forecast
            forecast = forecast[['ds', 'yhat']].tail(8)
            forecast['but_num_business_unit'] = but_num_business_unit
            forecast['dpt_num_department'] = dpt_num_department
            
            forecasts.append(forecast)

# Combine all forecasts into a single dataframe
forecast_df = pd.concat(forecasts)

# Merge the forecasts with the test dataset
test['week'] = pd.to_datetime(test['week'].astype(str) + '-1', format='%Y-%W-%w')
merged_test = test.merge(forecast_df, left_on=['but_num_business_unit', 'dpt_num_department', 'week'], right_on=['but_num_business_unit', 'dpt_num_department', 'ds'], how='left')

# Save the predictions to a file
merged_test[['but_num_business_unit', 'dpt_num_department', 'week', 'yhat']].to_csv('prophet_forecasts.csv', index=False)

print("Prophet forecasts saved to 'prophet_forecasts.csv'")

In [ ]:
# evaluation

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error


actual_turnover = test['actual_turnover']

# Calculate MAE and RMSE for Prophet forecast
mae_prophet = mean_absolute_error(actual_turnover, merged_test['yhat'])
rmse_prophet = np.sqrt(mean_squared_error(actual_turnover, merged_test['yhat']))

# Print the evaluation metrics
print(f"Prophet Forecast - MAE: {mae_prophet}, RMSE: {rmse_prophet}")
